#**Loading .xlsx type of file**

In [ ]:
from google.colab import files
uploaded = files.upload()




# **ge**

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:

import io
df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
df = df[['candidate', 'sentiment', 'text']]
print(df.head())

                candidate sentiment  \
0  No candidate mentioned   Neutral   
1            Scott Walker  Positive   
2  No candidate mentioned   Neutral   
3  No candidate mentioned  Positive   
4            Donald Trump  Positive   

                                                text  
0  RT @NancyLeeGrahn: How did everyone feel about...  
1  RT @ScottWalker: Didn't catch the full #GOPdeb...  
2  RT @TJMShow: No mention of Tamir Rice and the ...  
3  RT @RobGeorge: That Carly Fiorina is trending ...  
4  RT @DanScavino: #GOPDebate w/ @realDonaldTrump...  


# **Importing libraries that are needed to exicute codes.**

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
import io
import pandas as p



In [ ]:


stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text



In [ ]:
df['cleaned_text'] = df['text'].apply(clean_text)

max_words = 5000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df['cleaned_text'].values)
sequences = tokenizer.texts_to_sequences(df['cleaned_text'].values)
X = pad_sequences(sequences, maxlen=max_len)

le = LabelEncoder()
y = le.fit_transform(df['sentiment'])
y = tf.keras.utils.to_categorical(y)


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
print(model.summary())



In [ ]:
df['cleaned_text'] = df['text'].apply(clean_text)
print(df[['text', 'cleaned_text']].head())


In [ ]:
history = model.fit(
    X, y,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)

#Training Loss / Training Accuracy / Validation Loss / Validation Accuracy

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
print(X.shape)
print(X[0])


In [ ]:
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_data=(X_test, y_test))


In [ ]:

plt.figure(figsize=(10, 6))

plt.plot(history.history['accuracy'], marker='o')
plt.plot(history.history['val_accuracy'], marker='o')
plt.title('Model Accuracy', fontsize=16)
plt.ylabel('Accuracy', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.legend(['Train', 'Validation'], loc='lower right')
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 6))

plt.plot(history.history['loss'], marker='o')
plt.plot(history.history['val_loss'], marker='o')
plt.title('Model Loss', fontsize=16)
plt.ylabel('Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.legend(['Train', 'Validation'], loc='upper right')
plt.grid(True)
plt.tight_layout()
plt.show()



In [ ]:


y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
labels = ['Negative', 'Neutral', 'Positive']


fig, ax = plt.subplots(figsize=(8, 6))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot(cmap='Blues', ax=ax, colorbar=True)

plt.title('Confusion Matrix', fontsize=16)
plt.xlabel('Predicted Label', fontsize=14)
plt.ylabel('True Label', fontsize=14)
plt.grid(False)
plt.tight_layout()
plt.show()

#just fro visuals